In [1]:
import pandas as pd
import json
import requests
import time
from ApiKey import key1, key2
import datetime



In [2]:
baseurl = 'https://api.usa.gov/crime/fbi/sapi/'
agencyapi = 'api/agencies'

params = {'api_key' : key1}
x = 'Lee County'
y = x.replace(' County','')
print (y)
counties = pd.read_csv('bureau_of_labor_counties_with_data.csv')
counties['cname'] = counties['County Name'].str.replace(' County', '').str.upper()
counties.head()

Lee


,Area Type,Area Code,County Name,State,Display Level,Selectable,Sort Sequence,cname
0,F,CN0108100000000,Lee County,AL,0,T,71,LEE
1,F,CN0100700000000,Bibb County,AL,0,T,34,BIBB
2,F,CN0102700000000,Clay County,AL,0,T,44,CLAY
3,F,CN0104500000000,Dale County,AL,0,T,53,DALE
4,F,CN0106500000000,Hale County,AL,0,T,63,HALE


In [3]:
oris = requests.get(baseurl + agencyapi, params=params).json() #Retrieve all agencies from source
# print(json.dumps(oris, indent=2, sort_keys=True))

In [ ]:
# crimedata = requests.get(baseurl + '/api/summarized/agencies/AK0010200/offenses', params)
# crimedata
# mydat =

In [ ]:
# crimedata.json()

In [5]:
mydata = {'Agency':[], 'Year':[], 'Offense':[],'count':[], 'county':[]}
print(f"starting at {datetime.datetime.now()}")

for mystate in oris:
    print(mystate)
#     print(counties[counties['State'] == mystate]['State'].count())
    if counties[counties['State'] == mystate]['State'].count() > 0: #Check to see if state in county list
        statecounties = counties[counties['State'] == mystate]
#         print(mystate)
    #     print(oris[mystate])
        for myori in oris[mystate]:
            
#             print(statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']])
            if  statecounties[statecounties['cname'] ==  oris[mystate][myori]['county_name']]['cname'].count() > 0: #Check to see if county in list
                print('     ' + oris[mystate][myori]['county_name'])
                crimedata = ''        
                while crimedata == '':
                    try:
                        oriapi = f'/api/summarized/agencies/{myori}/offenses'
                        crimedata = requests.get(baseurl + oriapi, params).json()
                        crimedata['results']
                        for result in crimedata['results']:
                            mydata['Agency'].append(myori)
                            mydata['Year'].append(result['data_year'])
                            mydata['Offense'].append(result['offense'])
                            mydata['count'].append(result['actual'])
                            mydata['county'].append(oris[mystate][myori]['county_name'])        
                    except KeyError:
                        crimedate = ''
                        print('waiting 15 minutes')
                        print(datetime.datetime.now())
                        time.sleep(901)
                        if params['api_key'] == key1:
                            params['api_key'] = key2
                        else:
                            params['api_key'] = key1
                
            
        
        
            
        
        

starting at 2018-11-03 13:32:01.665848
HI
     HAWAII
     MAUI
     KAUAI
DE
     SUSSEX
     KENT
     SUSSEX
     SUSSEX
     SUSSEX
     KENT
     KENT
     SUSSEX
     NEW CASTLE
     NEW CASTLE
     KENT
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     SUSSEX
     KENT
     SUSSEX
     NEW CASTLE
     KENT
     SUSSEX
     SUSSEX
     KENT
     KENT
     KENT
     SUSSEX
     KENT
     NEW CASTLE
     SUSSEX
     SUSSEX
     SUSSEX
     SUSSEX
     KENT
     KENT
     SUSSEX
     SUSSEX
     NEW CASTLE
     KENT
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     NEW CASTLE
     SUSSEX
     SUSSEX
     KENT
     SUSSEX
     KENT
     KENT
     SUSSEX
     KENT
     SUSSEX
     KENT
     KENT
     SUSSEX
     KENT
     NEW CASTLE
     SUSSEX
     NEW CASTLE
     NEW CASTLE
PR
TX
     WASHINGTON
     BRAZORIA
     EDWARDS
     HARRIS
     MILAM
     MARION
     WILSON
     DENTON
     GALVESTON
     HARRIS
     BRAZORIA
     CALHOUN
     COLORADO
     HARRIS
  

KeyboardInterrupt: 

In [ ]:
# crimedata['results']
# 

In [ ]:
# agencies = pd.read_csv('./data/agencies.csv')
# # agencies.head()
# print(agencies.loc[agencies['agency_id']==61][['ncic_agency_name','agency_type_name', 'population']])
# agencies.columns
# agencies_rev=agencies[['agency_id','county_name']]
# agencies_rev

In [ ]:
# incidents = pd.read_csv('./data/NIBRS_incident.csv')
# incidents.head()
# incidents.dtypes
# incidents_rev = incidents[['agency_id','incident_date']]
# incidents_rev.dropna(axis=0, how='any',inplace=True)
# incidents_rev['AGENCY_ID'] =incidents_rev['agency_id'].astype('int64')
# incidents_rev.dtypes

In [ ]:
# merge_data = incidents_rev.merge(agencies_rev, left_on='AGENCY_ID', right_on='agency_id', how='inner')
# merge_data

In [ ]:
cdata = pd.DataFrame(mydata)
cdata.head()
crimegb = cdata.groupby(['Agency','Year','county']).sum()
crimegb.reset_index(inplace=True)
crimegb